# Financial and Economic Data Applications 
The use of Python in the financial industry has been increasing rapidly since 2005, led largely by the maturation of 
libararies and the availability of skilled python programmers. Institutions have found that Python is well-suited of 
as an interactive analysis environment as well as enabling robust systems to be developed often in a fraction of the time it would have taken in JAVA or C++... 

# Data Munging Topics

In [2]:
# Time Series and Cross-Section Alignment 
# On of the most time-consuming issues in working with the financial data is the so-called data alignment problem. 
# Two related time series may have indexes that don't line up perfectly or two DataFrame objects might have columns or row 
# labels that don't match. 

# Operations with Time Series of Different Frequencies
Economic time series often of annual, quarterly, and monthly frequencies. It is often necessary to perform operations.
Some are completely irregular, for example, earnings revisions for a stock may arrive at any time. 
The two main tools for frequency conversion and realignment are the 'resample' and 'reindex' methods. 
'resample' converts data to a fixed frequency while 'reindex' conforms data to a new index. Both support optional interpolation logic. 

In [3]:
# consider a small weekly time series of stock prices and volumes. 
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

# Let's create a small time series with weekly frequency. 
# The data is about the stock price and volume of a company.
# We will use the following data for our example.  

ts1 = Series(
    np.random.randn(3),
    index = pd.date_range('2012-6-13', periods = 3, freq = 'W-WED')
            )